<a href="https://colab.research.google.com/github/iwishkim/stock_market_analysis/blob/main/pykrx%EC%97%90%EC%BD%94%ED%94%84%EB%A1%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [pykrx](https://github.com/sharebook-kr/pykrx)
- [네이버증권](https://finance.naver.com/)

# 설치

In [ ]:
!pip install pykrx

In [ ]:
# (라이브러리 설치 후 런타임 재시작 필요)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 특정종목 주가 가져오기

In [ ]:
start_date = "20230101"
end_date = "20230422"
today = "20230422"
ticker = "086520" # 에코프로

In [ ]:
from pykrx import stock

df = stock.get_market_ohlcv(start_date, end_date, ticker)
df.head(2)

In [ ]:
df.tail(2)

In [ ]:
df.info()

In [ ]:
df['종가'].plot(figsize = (18, 5), title = '에코프로 주가 변동내역2023년1월부터')

In [ ]:
df['거래대금'].plot(figsize = (18, 5), title =('거래대금 변화'))

In [ ]:
df['거래량'].plot(figsize = (18, 5))

In [ ]:
import plotly.express as px
import plotly.graph_objs as go

chart_add=[
    go.Line(
        x=df.index, y=df['종가']),
    go.Bar(
        x=df.index, y=df['거래량'])]
layout = go.Layout(
    barmode='stack',
    title='에코프로 Stock Chart')
fig = go.Figure(chart_add, layout=layout)
fig.show()

In [ ]:
fig=px.scatter(df,x=df.index, y="종가", size="종가", hover_data=['종가']) #'],hover_data=df['Close'])
fig.show(0)

In [ ]:
fig = px.line(df, x=df.index, y=['종가','시가'])
fig.show()

In [ ]:
fig=px.bar(df, x=df.index, y=['종가','거래량'], title='에코프로 Stock Graph')
fig.show()

In [ ]:
fig = px.line(df, y='종가', title='Time Series with Rangeslider')

fig.update_xaxes(rangeslider_visible = True)
fig.show()

In [ ]:
fig = px.line(df, y='종가', title='Time Series with Rangeslider')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector = dict(
        buttons = list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

# 상관관계

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Matplotlib 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic')  

In [ ]:
df_corr = df.corr(method='pearson')
df_corr

In [ ]:
df['거래량'].corr(df['종가'])

In [ ]:
sns.pairplot(df_corr)
plt.show()

In [ ]:
sns.catplot(x='거래량', y='종가', hue='등락률', data=df_corr, kind='point')
plt.show()

In [ ]:
sns.heatmap(df_corr,
           annot=True,
           cmap='YlGn') # cmap color 참조
plt.show()

# 펀더멘탈

In [ ]:
df_fm = stock.get_market_fundamental(start_date, end_date, ticker)
df_fm.head(3)

In [ ]:
df_fm.tail()

In [ ]:
df_fm.info()

## 거래대금

## 일자별

In [ ]:
df_tvdate = stock.get_market_trading_value_by_date(start_date, end_date, ticker)
df_tvdate.head()

In [ ]:
df_tvdate.tail()

In [ ]:
df_tvdate_corr = df_tvdate.corr(method='pearson')
df_tvdate_corr

In [ ]:
sns.heatmap(df_tvdate_corr,
           annot=True,
           cmap='YlGn') # cmap color 참조
plt.show()

## 투자자별

In [ ]:
df_tviv = stock.get_market_trading_value_by_investor(start_date, end_date, ticker)
df_tviv.tail()

In [ ]:
# 공매도
df = stock.get_shorting_status_by_date("20230101", "202304122", "086520")
df.tail()

#이동평균 구하기

In [ ]:
from pykrx import stock

start_date = "20220101"
end_date = "20230422"
today = "20230422"
ticker = "086520" # 에코프로

df = stock.get_market_ohlcv(start_date, end_date, ticker)
df.head(3)

In [ ]:
df['ma5'] = df['종가'].rolling(5).mean()
df['ma20'] = df['종가'].rolling(20).mean()
df['ma100'] = df['종가'].rolling(100).mean()
df['ma200'] = df['종가'].rolling(200).mean()

In [ ]:
df.head()

In [ ]:
import plotly.express as px

df1 = df[['종가', 'ma5', 'ma20', 'ma100', 'ma200']]
fig = px.line(df1,  y=df1.columns,
              title='이동평균선')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")
fig.show()

#스토캐스틱

In [ ]:
# Fast %K = ((현재가 - n기간 중 최저가) / (n기간 중 최고가 - n기간 중 최저가)) * 100
def get_stochastic_fast_k(close_price, low, high, n=5):
  fast_k = ((close_price - low.rolling(n).min()) / (high.rolling(n).max() - low.rolling(n).min())) * 100
  return fast_k
 
# Slow %K = Fast %K의 m기간 이동평균
def get_stochastic_slow_k(fast_k, n=3):
  slow_k = fast_k.rolling(n).mean()
  return slow_k
 
# Slow %D = Slow %K의 t기간 이동평균
def get_stochastic_slow_d(slow_k, n=3):
  slow_d = slow_k.rolling(n).mean()
  return slow_d 

In [ ]:
# fast_k, slow_k, slow_d를 획득
df['fast_k'] = get_stochastic_fast_k(df['종가'], df['저가'], df['고가'], 5)
df['slow_k'] = get_stochastic_slow_k(df['fast_k'], 3)
df['slow_d'] = get_stochastic_slow_d(df['slow_k'], 3)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Line(x = df.index, y = df['종가'], name='종가'),
    secondary_y= True
)

# Add traces
fig.add_trace(
    go.Line(x = df.index, y = df['slow_k'], name='slow_k'),
    secondary_y= False
)

# Add traces
fig.add_trace(
    go.Line(x = df.index, y = df['slow_d'], name='slow_d'),
    secondary_y= False
)

# Add traces
fig.add_trace(
    go.Line(x = df.index, y=[80] * len(df.index), name='80선'),
    secondary_y= False
)

# Add traces
fig.add_trace(
    go.Line(x = df.index, y=[20] * len(df.index), name='20선'),
    secondary_y= False
)

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)

# Set y-axes titles
fig.update_yaxes(title_text="<b>stocastic</b> axis", secondary_y=False)
fig.update_yaxes(title_text="<b>종가</b> axis", secondary_y=True)

fig.show()

In [ ]:
# 골든크로스

df.query('(slow_k >= slow_d) and (slow_k <= 20)')

In [ ]:
# 데드클로스

df.query('(slow_k <= slow_d) and (slow_k >= 80)')